In [2]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [4]:
ticker = 'AMC'
date = '2017-05-01'
gap_perc_size = 1
def gap_play(ticker, date, gap_perc_size):
    data = web.DataReader(ticker, data_source='yahoo', start=date)
    data['Last Close'] = data['Close'].shift()
    del data['Close']
    del data['Adj Close']
    del data['Volume']
    data['gap size'] = (data['Open']-data['Last Close'])
    
    
    up_gap = data[data['gap size']>0]
    up_gap = up_gap.reset_index(drop=True)
    up_gap['return'] = (up_gap['Open']- up_gap['Low'])/up_gap['gap size']
    up_gap['return'] = np.clip(up_gap['return'], a_max=gap_perc_size, a_min=None)

    down_gap = data[data['gap size']<0]
    down_gap = down_gap.reset_index(drop=True)
    down_gap['return'] = (down_gap['Open']- down_gap['Low'])/down_gap['gap size']
    down_gap['return'] = np.clip(down_gap['return'], a_max=None, a_min=-1*gap_perc_size)

    res = {'Gap Ups': [up_gap['return'].mean(), up_gap['return'].std(), 
                       len(up_gap[up_gap['return']>=1])*100/len(up_gap), len(up_gap)],
          'Gap Downs': [down_gap['return'].mean(), down_gap['return'].std(), 
                        len(down_gap[down_gap['return']<=-1])*100/len(down_gap), len(down_gap)]}
    results = pd.DataFrame(data=res)
    results = results.rename(index = {0: 'Average Retracement', 1: '+/-', 2:'% Gaps closed', 3:'Number of Gaps'})
    return(results)

In [5]:
def div(x,y):
    return 0 if y==0 else x/y

def gap_test(ticker, date, gap_perc_size):
    data = web.DataReader(ticker, data_source='yahoo', start=date)
    data['Last Close'] = data['Close'].shift()
    del data['Close']
    del data['Adj Close']
    del data['Volume']
    data['gap size'] = (data['Open']-data['Last Close'])
    
    
    up_gap = data[data['gap size']*100 / data['Last Close'] > gap_perc_size]
    up_gap = up_gap.reset_index(drop=True)
    up_gap['return'] = (up_gap['Open']- up_gap['Low'])/up_gap['gap size']
    up_gap['return'] = np.clip(up_gap['return'], a_max=gap_perc_size, a_min=None)
    
    down_gap = data[data['gap size']*100 / data['Last Close'] < (-1)*gap_perc_size]
    down_gap = down_gap.reset_index(drop=True)
    down_gap['return'] = (down_gap['Open']- down_gap['High'])/down_gap['gap size']
    down_gap['return'] = np.clip(down_gap['return'], a_max=gap_perc_size, a_min=None)
    

    res = {'Gap Ups': [round(up_gap['return'].mean(), 2), round(up_gap['return'].std(), 2), 
                       round(div(len(up_gap[up_gap['return']>=1])*100,len(up_gap)), 2), int(len(up_gap))],
          'Gap Downs': [round(down_gap['return'].mean(), 2), round(down_gap['return'].std(), 2), 
                        round(div(len(down_gap[down_gap['return']>=1])*100, len(down_gap)),2), int(len(down_gap))]}
    
    results = pd.DataFrame(data=res)
    results = results.rename(index = {0: 'Average', 1: '+/-', 2:'% close', 3:'Number of Gaps'})
    
    return(results)    

In [6]:
def gap_test1(ticker, date, gap_perc_size):
    data = web.DataReader(ticker, data_source='yahoo', start=date)
    data['Last Close'] = data['Close'].shift()
    del data['Close']
    del data['Adj Close']
    del data['Volume']
    data['Gap'] = ((data['Open']-data['Last Close'])*100/data['Last Close'])
    #data = data[abs(data['Gap'])>gap_perc_size]
    
    
    up_gap = data[data['Gap']>gap_perc_size]
    up_gap = up_gap.reset_index(drop=True)
    up_gap['Size'] = up_gap['Open']-up_gap['Last Close']
    up_gap['return'] = (up_gap['Open']- up_gap['Low'])/up_gap['Size']
    up_gap['return'] = np.clip(up_gap['return'], a_max=gap_perc_size, a_min=None)
    
    down_gap = data[data['Gap']< -1*gap_perc_size]
    down_gap = down_gap.reset_index(drop=True)
    down_gap['Size'] = down_gap['Last Close'] - down_gap['Open']
    down_gap['return'] = (down_gap['Open'] - down_gap['High'])/down_gap['Size']
    down_gap['return'] = np.clip(down_gap['return'], a_max=None, a_min=(-1*gap_perc_size))
    
    
    return(up_gap['return'].mean(), down_gap['return'].mean())
        

In [7]:
ticker = 'TSLA'
date = '2017-05-01'
data = web.DataReader(ticker, data_source='yahoo', start=date)
#data['Date'] = data.index
#data['Date'] = pd.to_datetime(data['Date'])
#print(data)
data['Last Close'] = data['Close'].shift()
del data['Close']
del data['Adj Close']
del data['Volume']
data['Gap'] = ((data['Open']-data['Last Close'])*100/data['Last Close'])
data = data[abs(data['Gap'])>1]
up_gap = data[data['Gap']>1]
up_gap = up_gap.reset_index(drop=True)
up_gap['Size'] = up_gap['Open']-up_gap['Last Close']
up_gap['return'] = (up_gap['Open']- up_gap['Low'])/up_gap['Size']
up_gap['return'] = np.clip(up_gap['return'], a_max=1, a_min=None)
up_gap[up_gap['return'] == 1].sum(axis=0)
print(len(up_gap[up_gap['return']>=1])/len(up_gap), up_gap['return'].mean())
down_gap = data[data['Gap']<1]
down_gap = down_gap.reset_index(drop=True)
down_gap['Size'] = down_gap['Last Close'] - down_gap['Open']
down_gap['return'] = (down_gap['Open'] - down_gap['High'])/down_gap['Size']
down_gap['return'] = np.clip(down_gap['return'], a_max=None, a_min=-1)
print(down_gap)

0.39574468085106385 0.7072696950737902
           High         Low        Open  Last Close       Gap       Size  \
0     61.554001   58.152000   61.487999   62.203999 -1.151051   0.716000   
1     64.040001   62.506001   63.675999   64.961998 -1.979618   1.285999   
2     75.092003   73.297997   74.500000   76.132004 -2.143650   1.632004   
3     69.447998   65.265999   69.440002   70.524002 -1.537065   1.084000   
4     64.157997   61.259998   63.452000   65.417999 -3.005288   1.966000   
..          ...         ...         ...         ...       ...        ...   
174  854.429993  803.619995  849.400024  880.020020 -3.479466  30.619995   
175  863.000000  838.750000  843.390015  854.409973 -1.289774  11.019958   
176  848.000000  828.619995  834.309998  844.989990 -1.263919  10.679993   
177  891.500000  858.659973  870.349976  883.090027 -1.442667  12.740051   
178  847.996582  801.000000  820.000000  864.159973 -5.110162  44.159973   

       return  
0   -0.092181  
1   -0.283050  


In [8]:
def gap_test2(ticker, date, gap_perc_size):
    data = web.DataReader(ticker, data_source='yahoo', start=date)
    data['Last Close'] = data['Close'].shift()
    del data['Close']
    del data['Adj Close']
    del data['Volume']
    data['Gap'] = ((data['Open']-data['Last Close'])*100/data['Last Close'])
    #data = data[abs(data['Gap'])>gap_perc_size]
    
    
    up_gap = data[data['Gap']>gap_perc_size]
    up_gap = up_gap.reset_index(drop=True)
    up_gap['Size'] = up_gap['Open']-up_gap['Last Close']
    up_gap['return'] = (up_gap['Open']- up_gap['Low'])/up_gap['Size']
    up_gap['return'] = np.clip(up_gap['return'], a_max=gap_perc_size, a_min=None)
    
    
    down_gap = data[data['Gap']< -1*gap_perc_size]
    down_gap = down_gap.reset_index(drop=True)
    down_gap['Size'] = down_gap['Last Close'] - down_gap['Open']
    down_gap['return'] = (down_gap['High'] - down_gap['Open'])/down_gap['Size']
    down_gap['return'] = np.clip(down_gap['return'], a_max=None, a_min=(-1*gap_perc_size))
    
    
    res = {'Gap Ups': [up_gap['return'].mean(), up_gap['return'].std(), len(up_gap[up_gap['return']>=1])*100/len(up_gap)],
           'Gap Downs': [down_gap['return'].mean(), down_gap['return'].std(), len(down_gap[down_gap['return']<=1])*100/len(down_gap)]}
    
    results = pd.DataFrame(data=res)
    results = results.rename(index = {0: 'Average', 1: '+/-', 2:'% close'})
    
    return(results)
        

In [9]:
print(gap_play('TSLA', '2017-05-01', 1))
print(gap_test('TSLA', '2017-05-01', 1))
print(gap_test1('TSLA', '2017-05-01', 1))
print(gap_test2('TSLA', '2017-05-01', 1))

                        Gap Ups   Gap Downs
Average Retracement    0.823867   -0.806447
+/-                    0.290503    0.304429
% Gaps closed         63.465160   61.463415
Number of Gaps       531.000000  410.000000
                Gap Ups  Gap Downs
Average            0.71       0.70
+/-                0.33       0.34
% close           39.57      43.58
Number of Gaps   235.00     179.00
(0.7072696950737902, -0.6994781613079908)
           Gap Ups  Gap Downs
Average   0.707270   1.151492
+/-       0.328701   1.075390
% close  39.574468  56.424581


In [10]:
print(gap_play('AMC', '2021-01-26', 1))
print(gap_test('AMC', '2021-01-26', 1))
print(gap_test1('AMC', '2021-01-26', 1))
print(gap_test2('AMC', '2021-01-26', 1)) # incorrect % close value (down)

                      Gap Ups  Gap Downs
Average Retracement  0.606632  -0.690656
+/-                       NaN        NaN
% Gaps closed        0.000000   0.000000
Number of Gaps       1.000000   1.000000
                Gap Ups  Gap Downs
Average            0.61       0.57
+/-                 NaN        NaN
% close            0.00       0.00
Number of Gaps     1.00       1.00
(0.6066319801224014, -0.5707071230078502)
          Gap Ups   Gap Downs
Average  0.606632    0.570707
+/-           NaN         NaN
% close  0.000000  100.000000


In [3]:
b = web.DataReader('AMC','yahoo', start='2021-01-26')
print(b)

                 High    Low   Open  Close      Volume  Adj Close
Date                                                             
2021-01-26   5.190000   4.37   5.09   4.96   456850200       4.96
2021-01-27  20.360001  11.01  20.34  19.90  1222342500      19.90
2021-01-28  16.500000   6.51  11.98   8.91   475504650       8.91
